In [ ]:
# %pip install --upgrade domolibrary

In [6]:
DATASET_ID = 'e1b88aab-42b5-4733-9817-cae8937ecb08' # cnn world

In [4]:
import domolibrary.client.DomoAuth as dmda
import domojupyter as dj


async def generate_domo_auth(account_name : str, 
                             domo_instance : str,
                             debug_prn: bool = False) -> dmda.DomoAuth:
    
    account_properties = dj.get_account_property_keys(account_name)
    if debug_prn:
        print(account_properties)
    
    creds = {prop : dj.get_account_property_value(account_name, prop) for prop in account_properties}

    auth = dmda.DomoTokenAuth(domo_access_token = creds['domoAccessToken'],
                   domo_instance = domo_instance)

    if debug_prn:
        print((await auth.who_am_i()).response)
    
    return auth

auth = await generate_domo_auth( 'sdk_domo-community', 'domo_community', debug_prn = False)
# auth.generate_auth_header()

In [7]:
import domolibrary.classes.DomoDataset as dmds
from pprint import pprint

async def get_dataset_and_schema(auth, dataset_id, debug_api : bool = False):

    domo_dataset = await dmds.DomoDataset.get_by_id(dataset_id = dataset_id, 
                                                    auth = auth, 
                                                    debug_api = debug_api)
    
    await domo_dataset.Schema.get(debug_api = debug_api)
    
        
    return domo_dataset

domo_dataset = await get_dataset_and_schema(auth = auth, 
                                            dataset_id = DATASET_ID,
                                            debug_api = True
                                           )

domo_dataset

🐛 debugging get_data
{'body': None,
 'function_name': 'get_traceback',
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': '7b92f788635a46b82f914b8ef63daf8658fcbf0da9eabc8b'},
 'method': 'GET',
 'params': None,
 'parent_class': 'DomoDataset',
 'url': 'https://domo_community.domo.com/api/data/v3/datasources/e1b88aab-42b5-4733-9817-cae8937ecb08'}
get_data: no body
get_data_response <Response [200 OK]>
🐛 debugging get_data
{'body': None,
 'function_name': 'get_traceback',
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': '7b92f788635a46b82f914b8ef63daf8658fcbf0da9eabc8b'},
 'method': 'GET',
 'params': None,
 'parent_class': None,
 'url': 'https://domo_community.domo.com/api/query/v1/datasources/e1b88aab-42b5-4733-9817-cae8937ec

DomoDataset(id='e1b88aab-42b5-4733-9817-cae8937ecb08', display_type='cnn', data_provider_type='cnn', name='cnn_world', description='', row_count=69, column_count=8, stream_id=1934, owner={'id': '1893952720', 'name': 'Jae Wilson1', 'type': 'USER', 'group': False}, formula={}, Schema=DomoDataset_Schema(dataset_id='e1b88aab-42b5-4733-9817-cae8937ecb08', columns=[DomoDataset_Schema_Column(name='Title', id='Title', type='STRING', order=0, visible=True, upsert_key=False), DomoDataset_Schema_Column(name='Guid', id='Guid', type='STRING', order=1, visible=True, upsert_key=False), DomoDataset_Schema_Column(name='Link', id='Link', type='STRING', order=2, visible=True, upsert_key=False), DomoDataset_Schema_Column(name='Description', id='Description', type='STRING', order=3, visible=True, upsert_key=False), DomoDataset_Schema_Column(name='PublicationDate', id='PublicationDate', type='DATETIME', order=4, visible=True, upsert_key=False), DomoDataset_Schema_Column(name='FeedburnerOriginalLink', id='Fe

In [8]:
import pandas as pd

pd.DataFrame(domo_dataset.Schema.columns)

,name,id,type,order,visible,upsert_key
0,Title,Title,STRING,0,True,False
1,Guid,Guid,STRING,1,True,False
2,Link,Link,STRING,2,True,False
3,Description,Description,STRING,3,True,False
4,PublicationDate,PublicationDate,DATETIME,4,True,False
5,FeedburnerOriginalLink,FeedburnerOriginalLink,STRING,5,True,False
6,_BATCH_ID_,_BATCH_ID_,DOUBLE,6,True,False
7,_BATCH_LAST_RUN_,_BATCH_LAST_RUN_,DATETIME,7,True,False


In [12]:
import pandas as pd
import datetime as dt
sample_df = pd.DataFrame(columns = ["dataset_id", "id", "description"], data = [[DATASET_ID, "Title", f"description of the article - {dt.datetime.now().time()}"],
                                                                                [DATASET_ID, "PublicationDate" , "first time published"],
                                                                                ['f02bb1f9-0e06-40cf-a621-006482a6fc3e', 'comment', 'hello world']]
                        )

sample_df

,dataset_id,id,description
0,e1b88aab-42b5-4733-9817-cae8937ecb08,Title,description of the article - 03:17:23.588655
1,e1b88aab-42b5-4733-9817-cae8937ecb08,PublicationDate,first time published
2,f02bb1f9-0e06-40cf-a621-006482a6fc3e,comment,hello world


In [13]:
def update_schema_description_text(col_id :str , 
                                   new_description : str , 
                                   schema_columns):
    for column in schema_columns:
        if column.id != col_id:
            continue 
        
        column.description = new_description
        

for index, row in sample_df.iterrows() :
    if row.dataset_id != DATASET_ID:
        continue
    update_schema_description_text(col_id = row['id'], 
                                   new_description = row['description'], 
                                   schema_columns = domo_dataset.Schema.columns) 


pd.DataFrame((await domo_dataset.Schema.alter_schema_descriptions(debug_api = True)).response.get('columns'))

🐛 debugging get_data
{'body': {'columns': [{'description': 'description of the article - '
                                      '03:17:23.588655',
                       'id': 'Title',
                       'name': 'Title',
                       'order': 0,
                       'type': 'STRING',
                       'upsertKey': False,
                       'visible': True},
                      {'id': 'Guid',
                       'name': 'Guid',
                       'order': 1,
                       'type': 'STRING',
                       'upsertKey': False,
                       'visible': True},
                      {'id': 'Link',
                       'name': 'Link',
                       'order': 2,
                       'type': 'STRING',
                       'upsertKey': False,
                       'visible': True},
                      {'id': 'Description',
                       'name': 'Description',
                       'order': 3,
                 

,name,newName,description,tags,visible,remove,type,formula,calculatedExpr,calculated,order
0,Title,None,description of the article - 03:17:23.588655,None,True,False,STRING,None,None,False,0
1,Guid,None,,None,True,False,STRING,None,None,False,1
2,Link,None,,None,True,False,STRING,None,None,False,2
3,Description,None,,None,True,False,STRING,None,None,False,3
4,PublicationDate,None,first time published,None,True,False,DATETIME,None,None,False,4
5,FeedburnerOriginalLink,None,,None,True,False,STRING,None,None,False,5
6,_BATCH_ID_,None,,None,True,False,DOUBLE,None,None,False,6
7,_BATCH_LAST_RUN_,None,,None,True,False,DATETIME,None,None,False,7


In [17]:
from pprint import pprint

def generate_config_dataset(df, debug_prn: bool = False):
    ls =  df.to_dict( orient = 'records')
    
    if debug_prn:
        pprint(ls)
    
    res = {}
    
    for index , row in enumerate(ls):
        dataset_id = row.pop('dataset_id')
        
        if not res.get(dataset_id):
            res.update({dataset_id :  []})
        
        res[dataset_id].append(row)
    
    return res

    

config_ls = generate_config_dataset(sample_df, debug_prn = True)
pprint(config_ls)

[{'dataset_id': 'e1b88aab-42b5-4733-9817-cae8937ecb08',
  'description': 'description of the article - 03:17:23.588655',
  'id': 'Title'},
 {'dataset_id': 'e1b88aab-42b5-4733-9817-cae8937ecb08',
  'description': 'first time published',
  'id': 'PublicationDate'},
 {'dataset_id': 'f02bb1f9-0e06-40cf-a621-006482a6fc3e',
  'description': 'hello world',
  'id': 'comment'}]
{'e1b88aab-42b5-4733-9817-cae8937ecb08': [{'description': 'description of the '
                                                          'article - '
                                                          '03:17:23.588655',
                                           'id': 'Title'},
                                          {'description': 'first time '
                                                          'published',
                                           'id': 'PublicationDate'}],
 'f02bb1f9-0e06-40cf-a621-006482a6fc3e': [{'description': 'hello world',
                                           'id': 'comme

In [20]:
from typing import List
import domolibrary.utils.chunk_execution as dmce


async def update_schema(dataset_id : str, 
                        new_descriptions : List[dict], 
                        auth : dmda.DomoAuth,
                        debug_api : bool = False
                       ):
    
    domo_dataset = await get_dataset_and_schema(auth, dataset_id)
    
    for index, row in enumerate(new_descriptions):        
        update_schema_description_text(col_id = row['id'], 
                                       new_description = row['description'], 
                                       schema_columns = domo_dataset.Schema.columns) 


    res  =await domo_dataset.Schema.alter_schema_descriptions(debug_api = debug_api)
    
    return pd.DataFrame(res.response.get('columns'))
    
    
dfs = await dmce.gather_with_concurrency(*[
    update_schema(dataset_id = dataset_id,
                  new_descriptions = new_descriptions,
                  auth = auth) for dataset_id, new_descriptions in config_ls.items()], n = 10)

pd.concat(dfs)

,name,newName,description,tags,visible,remove,type,formula,calculatedExpr,calculated,order
0,Title,None,description of the article - 03:17:23.588655,None,True,False,STRING,None,None,False,0
1,Guid,None,,None,True,False,STRING,None,None,False,1
2,Link,None,,None,True,False,STRING,None,None,False,2
3,Description,None,,None,True,False,STRING,None,None,False,3
4,PublicationDate,None,first time published,None,True,False,DATETIME,None,None,False,4
5,FeedburnerOriginalLink,None,,None,True,False,STRING,None,None,False,5
6,_BATCH_ID_,None,,None,True,False,DOUBLE,None,None,False,6
7,_BATCH_LAST_RUN_,None,,None,True,False,DATETIME,None,None,False,7
0,comment,None,hello world,None,True,False,STRING,None,None,False,0
1,__documentId__,None,,None,True,False,STRING,None,None,False,1
